## Google Analytics Customer Revenue Prediction

The goal is to analyze a Google Merchandise Store (also known as GStore, where Google swag is sold) customer dataset to predict revenue per customer. 

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import gc
import sys
import math

from pandas.io.json import json_normalize
from datetime import datetime

In [ ]:
# Enable garbage collection
gc.enable()

# Define the list of features to extract from the dataset
features = ['channelGrouping', 'date', 'fullVisitorId', 'visitId', 'visitNumber', 'visitStartTime', 'device.browser', 'device.deviceCategory', 'device.isMobile', 'device.operatingSystem', 'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region', 'geoNetwork.subContinent', 'totals.bounces', 'totals.hits', 'totals.newVisits', 'totals.pageviews', 'totals.transactionRevenue', 'trafficSource.adContent', 'trafficSource.campaign', 'trafficSource.isTrueDirect', 'trafficSource.keyword', 'trafficSource.medium', 'trafficSource.referralPath', 'trafficSource.source', 'customDimensions']

# Function to load and preprocess the CSV data
def load_df(csv_path):
    # Define the columns with JSON data
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    # Create an empty DataFrame to store the data
    ans = pd.DataFrame()
    
    # Read the CSV file in chunks
    dfs = pd.read_csv(csv_path, sep=',', converters={column: json.loads for column in JSON_COLUMNS}, dtype={'fullVisitorId': 'str'}, chunksize=100000)
    
    # Process each chunk of data
    for df in dfs:
        df.reset_index(drop=True, inplace=True)
        
        # Normalize the JSON columns and merge with the main DataFrame
        for column in JSON_COLUMNS:
            column_as_df = json_normalize(df[column])
            column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
            df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

        use_df = df[features]
        del df
        gc.collect()
        
        # Concatenate the processed chunk to the final DataFrame
        ans = pd.concat([ans, use_df], axis=0).reset_index(drop=True)
        
    return ans

In [ ]:
%%time
train = load_df('../root/input/train_v2.csv')
test = load_df('../root/input/test_v2.csv')

print('train date:', min(train['date']), 'to', max(train['date']))
print('test date:', min(test['date']), 'to', max(test['date']))

In [ ]:
def reduce_mem_usage(df):
    """
    Iterate through all the columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    # Calculate the initial memory usage of the dataframe
    start_mem = df.memory_usage().sum()
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    # Iterate through each column
    for col in df.columns:
        col_type = df[col].dtype
        
        # Convert numeric columns to smaller data types
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    
    # Calculate the final memory usage of the dataframe after optimization
    end_mem = df.memory_usage().sum()
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

# Apply memory optimization to the 'train' and 'test' dataframes
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)


## Data Preprocessing + EDA

In [ ]:
for c in train.columns.values:
    if c not in test.columns.values: print(c)

In [ ]:
# Fill missing values in 'totals.transactionRevenue' column with 0
train['totals.transactionRevenue'].fillna(0, inplace=True)

# Convert 'totals.transactionRevenue' column to log scale using np.log1p
train['totals.transactionRevenue'] = np.log1p(train['totals.transactionRevenue'].astype(float))

# Display descriptive statistics of the 'totals.transactionRevenue' column
print(train['totals.transactionRevenue'].describe())

In [ ]:
test['totals.transactionRevenue'] = np.nan

In [ ]:
all_data = train.append(test, sort=False).reset_index(drop=True)

In [ ]:
print(all_data.info())

In [ ]:
# Calculate the count of null values in each column and sort them in ascending order
null_cnt = train.isnull().sum().sort_values()

# Print the columns with null values greater than 0
print(null_cnt[null_cnt > 0])

In [ ]:
# Fill missing values in object features with 'unknown'
for col in ['trafficSource.keyword', 'trafficSource.referralPath', 'trafficSource.adContent']:
    all_data[col].fillna('unknown', inplace=True)

# Fill missing values in numeric features
all_data['totals.pageviews'].fillna(1, inplace=True)
all_data['totals.newVisits'].fillna(0, inplace=True)
all_data['totals.bounces'].fillna(0, inplace=True)
all_data['totals.pageviews'] = all_data['totals.pageviews'].astype(int)
all_data['totals.newVisits'] = all_data['totals.newVisits'].astype(int)
all_data['totals.bounces'] = all_data['totals.bounces'].astype(int)

# Fill missing values in boolean feature with False
all_data['trafficSource.isTrueDirect'].fillna(False, inplace=True)

In [ ]:
# Find constant columns (columns with only one unique value)
constant_column = [col for col in all_data.columns if all_data[col].nunique() == 1]

# Drop the constant columns from the dataframe
print('drop columns:', constant_column)
all_data.drop(constant_column, axis=1, inplace=True)

In [ ]:
# Select a specific visitor using their fullVisitorId and sort the data by visitNumber
visitor_data = all_data[all_data['fullVisitorId'] == '7813149961404844386'].sort_values(by='visitNumber')

# Display the selected columns for the visitor
visitor_data[['date', 'visitId', 'visitNumber', 'totals.hits', 'totals.pageviews']].head(20)

In [ ]:
# Filter the train dataframe to include only rows where 'totals.transactionRevenue' is greater than 0
train_rev = train[train['totals.transactionRevenue'] > 0].copy()

# Print the length of the filtered dataframe
print(len(train_rev))

# Display the first few rows of the filtered dataframe
train_rev.head()

In [ ]:
def plotCategoryRateBar(a, b, colName, topN=np.nan):
    """
    Plot the category rate bar chart comparing the distribution of a categorical column
    between two datasets 'a' and 'b'.

    Parameters:
    - a: DataFrame representing the first dataset.
    - b: DataFrame representing the second dataset.
    - colName: Name of the categorical column to compare.
    - topN: Number of top categories to consider (optional).

    Returns:
    None
    """
    if topN == topN:  # Check if topN is not NaN
        vals = b[colName].value_counts()[:topN]
        subA = a.loc[a[colName].isin(vals.index.values), colName]
        df = pd.DataFrame({'All': subA.value_counts() / len(a), 'Revenue': vals / len(b)})
    else:
        df = pd.DataFrame({'All': a[colName].value_counts() / len(a), 'Revenue': b[colName].value_counts() / len(b)})
    
    # Plot the horizontal bar chart
    df.sort_values('Revenue').plot.barh(colormap='jet')

### customDimensions

In [ ]:
# Print the count of unique customDimensions
print('unique customDimensions count:', train['customDimensions'].nunique())

# Plot the category rate bar chart for 'customDimensions'
plotCategoryRateBar(all_data, train_rev, 'customDimensions')

### date

In [ ]:
# Define the date format
format_str = '%Y%m%d'

# Convert the 'date' column to a datetime format
all_data['formated_date'] = all_data['date'].apply(lambda x: datetime.strptime(str(x), format_str))

# Extract year, month, quarter of the month, day, and weekday from the 'formated_date' column
all_data['_year'] = all_data['formated_date'].apply(lambda x: x.year)
all_data['_month'] = all_data['formated_date'].apply(lambda x: x.month)
all_data['_quarterMonth'] = all_data['formated_date'].apply(lambda x: x.day // 8)
all_data['_day'] = all_data['formated_date'].apply(lambda x: x.day)
all_data['_weekday'] = all_data['formated_date'].apply(lambda x: x.weekday())

# Drop the 'date' and 'formated_date' columns from the dataframe
all_data.drop(['date', 'formated_date'], axis=1, inplace=True)

### newVisits

In [ ]:
# Print the total count and unique count of 'fullVisitorId' and 'visitId' columns in the train dataset
print('train all:', len(train))
print('train unique fullVisitorId:', train['fullVisitorId'].nunique())
print('train unique visitId:', train['visitId'].nunique())
print('-' * 30)

# Print the total count and unique count of 'fullVisitorId' and 'visitId' columns in the test dataset
print('test all:', len(test))
print('test unique fullVisitorId:', test['fullVisitorId'].nunique())
print('test unique visitId:', test['visitId'].nunique())

In [ ]:
# Print the top 5 most frequent visit numbers
print(all_data['visitNumber'].value_counts()[:5])
print('-' * 30)

# Print the count of new visits and non-new visits
print(all_data['totals.newVisits'].value_counts())
print('-' * 30)

# Print the count of bounces and non-bounces
print(all_data['totals.bounces'].value_counts())

In [ ]:
# Extract the hour from 'visitStartTime' and create a new column '_visitStartHour'
all_data['_visitStartHour'] = all_data['visitStartTime'].apply(lambda x: str(datetime.fromtimestamp(x).hour))

### device

In [ ]:
print('unique browser count:', train['device.browser'].nunique())
plotCategoryRateBar(all_data, train_rev, 'device.browser', 10)

In [ ]:
pd.crosstab(all_data['device.deviceCategory'], all_data['device.isMobile'], margins=False)

all_data['isMobile'] = True
all_data.loc[all_data['device.deviceCategory'] == 'desktop', 'isMobile'] = False

In [ ]:
print('unique operatingSystem count:', train['device.operatingSystem'].nunique())
plotCategoryRateBar(all_data, train_rev, 'device.operatingSystem', 10)

### geoNetwork

In [ ]:
print('unique geoNetwork.city count:', train['geoNetwork.city'].nunique())
plotCategoryRateBar(all_data, train_rev, 'geoNetwork.city', 10)

In [ ]:
print('unique geoNetwork.region count:', train['geoNetwork.region'].nunique())
plotCategoryRateBar(all_data, train_rev, 'geoNetwork.region', 10)

In [ ]:
print('unique geoNetwork.subContinent count:', train['geoNetwork.subContinent'].nunique())
plotCategoryRateBar(all_data, train_rev, 'geoNetwork.subContinent', 10)

In [ ]:
print('unique geoNetwork.continent count:', train['geoNetwork.continent'].nunique())
plotCategoryRateBar(all_data, train_rev, 'geoNetwork.continent')

In [ ]:
print('unique geoNetwork.metro count:', train['geoNetwork.metro'].nunique())
plotCategoryRateBar(all_data, train_rev, 'geoNetwork.metro', 10)

In [ ]:
print('unique geoNetwork.networkDomain count:', train['geoNetwork.networkDomain'].nunique())
plotCategoryRateBar(all_data, train_rev, 'geoNetwork.networkDomain', 10)

### totals

In [ ]:
print(all_data['totals.hits'].value_counts()[:10])

all_data['totals.hits'] = all_data['totals.hits'].astype(int)

In [ ]:
print(all_data['totals.pageviews'].value_counts()[:10])

all_data['totals.pageviews'] = all_data['totals.pageviews'].astype(int)

### trafficSource

In [ ]:
print('unique trafficSource.adContent count:', train['trafficSource.adContent'].nunique())

plotCategoryRateBar(all_data, train_rev, 'trafficSource.adContent', 10)

all_data['_adContentGMC'] = (all_data['trafficSource.adContent'] == 'Google Merchandise Collection').astype(np.uint8)

In [ ]:
print('unique trafficSource.campaign count:', train['trafficSource.campaign'].nunique())
plotCategoryRateBar(all_data, train_rev, 'trafficSource.campaign', 10)

all_data['_withCampaign'] = (all_data['trafficSource.campaign'] != '(not set)').astype(np.uint8)

In [ ]:
print(all_data['trafficSource.isTrueDirect'].value_counts())
plotCategoryRateBar(all_data, train_rev, 'trafficSource.isTrueDirect')

In [ ]:
print('unique trafficSource.keyword count:', train['trafficSource.keyword'].nunique())
plotCategoryRateBar(all_data, train_rev, 'trafficSource.keyword', 10)

In [ ]:
print('unique trafficSource.medium count:', train['trafficSource.medium'].nunique())
plotCategoryRateBar(all_data, train_rev, 'trafficSource.medium')

In [ ]:
print('unique trafficSource.referralPath count:', train['trafficSource.referralPath'].nunique())
plotCategoryRateBar(all_data, train_rev, 'trafficSource.referralPath', 10)

all_data['_referralRoot'] = (all_data['trafficSource.referralPath'] == '/').astype(np.uint8)

In [ ]:
print('unique trafficSource.source count:', train['trafficSource.source'].nunique())
plotCategoryRateBar(all_data, train_rev, 'trafficSource.source', 10)

all_data['_sourceGpmall'] = (all_data['trafficSource.source'] == 'mall.googleplex.com').astype(np.uint8)

### Aggregate

In [ ]:
_='''
'''

# Calculate mean and sum of 'totals.hits' per day, weekday, and month
all_data['_meanHitsPerDay'] = all_data.groupby(['_day'])['totals.hits'].transform('mean')
all_data['_meanHitsPerWeekday'] = all_data.groupby(['_weekday'])['totals.hits'].transform('mean')
all_data['_meanHitsPerMonth'] = all_data.groupby(['_month'])['totals.hits'].transform('mean')
all_data['_sumHitsPerDay'] = all_data.groupby(['_day'])['totals.hits'].transform('sum')
all_data['_sumHitsPerWeekday'] = all_data.groupby(['_weekday'])['totals.hits'].transform('sum')
all_data['_sumHitsPerMonth'] = all_data.groupby(['_month'])['totals.hits'].transform('sum')

# Calculate the mean of 'totals.hits' and 'totals.pageviews' per user
for feature in ['totals.hits', 'totals.pageviews']:
    info = all_data.groupby('fullVisitorId')[feature].mean()
    all_data['_usermean_' + feature] = all_data.fullVisitorId.map(info)

# Calculate the maximum 'visitNumber' per user    
for feature in ['visitNumber']:
    info = all_data.groupby('fullVisitorId')[feature].max()
    all_data['_usermax_' + feature] = all_data.fullVisitorId.map(info)

# Clean up the temporary variable
del info

In [ ]:
all_data['_source.country'] = all_data['trafficSource.source'] + '_' + all_data['geoNetwork.country']
all_data['_campaign.medium'] = all_data['trafficSource.campaign'] + '_' + all_data['trafficSource.medium']
all_data['_browser.category'] = all_data['device.browser'] + '_' + all_data['device.deviceCategory']
all_data['_browser.os'] = all_data['device.browser'] + '_' + all_data['device.operatingSystem']

### Select feature

In [ ]:
null_cnt = all_data.isnull().sum().sort_values()
print(null_cnt[null_cnt > 0])

In [ ]:
# Drop the 'visitId' and 'visitStartTime' columns from the dataframe
all_data.drop(['visitId', 'visitStartTime'], axis=1, inplace=True)

# Factorize object columns and fill missing values with 'unknown'
for i, t in all_data.loc[:, all_data.columns != 'fullVisitorId'].dtypes.iteritems():
    if t == object:
        all_data[i].fillna('unknown', inplace=True)
        all_data[i] = pd.factorize(all_data[i])[0]

## Modelling

In [ ]:
all_data.info()

In [ ]:
# Create the train and test datasets based on the presence of transaction revenue
train = all_data[all_data['totals.transactionRevenue'].notnull()]
test = all_data[all_data['totals.transactionRevenue'].isnull()].drop(['totals.transactionRevenue'], axis=1)

In [ ]:
test.shape

In [ ]:
# Store the 'fullVisitorId' column in train and test datasets
train_id = train['fullVisitorId']
test_id = test['fullVisitorId']

# Remove the 'totals.transactionRevenue' column from train dataset and assign it to Y_train_reg
Y_train_reg = train.pop('totals.transactionRevenue')

# Remove the 'fullVisitorId' column from X_train and X_test datasets
X_train = train.drop(['fullVisitorId'], axis=1)
X_test = test.drop(['fullVisitorId'], axis=1)

# Print the shapes of X_train and X_test datasets
print(X_train.shape, X_test.shape)

In [ ]:
# Clear memory and garbage collect
del all_data, train, test, train_rev
gc.collect()

# Print the sizes of the variables
print(pd.DataFrame([[val for val in dir()], [sys.getsizeof(eval(val)) for val in dir()]],
                   index=['name', 'size']).T.sort_values('size', ascending=False).reset_index(drop=True)[:10])

In [ ]:
from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

In [ ]:
# Define the parameters for the LGBMRegressor
params = {'learning_rate': 0.01,
          'objective': 'regression',
          'metric': 'rmse',
          'num_leaves': 31,
          'verbose': 1,
          'random_state': 42,
          'bagging_fraction': 0.6,
          'feature_fraction': 0.6}

# Initialize the GroupKFold with 5 splits
folds = GroupKFold(n_splits=5)

# Initialize arrays for out-of-fold predictions and submission predictions
oof_preds = np.zeros(X_train.shape[0])
sub_preds = np.zeros(X_test.shape[0])

# Iterate over the folds and train the LGBMRegressor
for fold_, (trn_, val_) in enumerate(folds.split(X_train, Y_train_reg, groups=train_id)):
    trn_x, trn_y = X_train.iloc[trn_], Y_train_reg.iloc[trn_]
    val_x, val_y = X_train.iloc[val_], Y_train_reg.iloc[val_]
    
    # Initialize and train the LGBMRegressor
    reg = lgb.LGBMRegressor(**params, n_estimators=3000)
    reg.fit(trn_x, trn_y, eval_set=[(val_x, val_y)], early_stopping_rounds=50, verbose=500)
    
    # Make predictions for the validation set and test set
    oof_preds[val_] = reg.predict(val_x, num_iteration=reg.best_iteration_)
    sub_preds += reg.predict(X_test, num_iteration=reg.best_iteration_) / folds.n_splits

# Store the final predictions
pred = sub_preds

In [ ]:
# Calculate feature importance
feature_importance = reg.feature_importances_
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
sorted_idx = sorted_idx[len(feature_importance) - 30:]
pos = np.arange(sorted_idx.shape[0]) + .5

# Plot feature importance
plt.figure(figsize=(12, 8))
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, X_train.columns[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()

In [ ]:
# Create a submission DataFrame with fullVisitorId and predicted log revenue
submission = pd.DataFrame({'fullVisitorId': test_id, 'PredictedLogRevenue': pred})

# Apply transformations to predicted log revenue
submission["PredictedLogRevenue"] = np.expm1(submission["PredictedLogRevenue"])
submission["PredictedLogRevenue"] = submission["PredictedLogRevenue"].apply(lambda x: 0.0 if x < 0 else x)
submission["PredictedLogRevenue"] = submission["PredictedLogRevenue"].fillna(0.0)

# Aggregate the predicted log revenue by fullVisitorId
submission_sum = submission[['fullVisitorId', 'PredictedLogRevenue']].groupby('fullVisitorId').sum().reset_index()
submission_sum["PredictedLogRevenue"] = np.log1p(submission_sum["PredictedLogRevenue"])

# Save the submission as a CSV file
submission_sum.to_csv("submission.csv", index=False)
submission_sum.head(20)